In [3]:
# imports iniciales
import pandas as pd
import numpy as np
import datetime
from datetime import datetime, timedelta
import calendar
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)

In [4]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
import numpy as np
import datetime
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import metrics
import warnings
warnings.filterwarnings('ignore')

In [5]:
########## https://jessesw.com/XG-Boost/

In [6]:
training_labels = pd.read_csv("labels_training_set.csv", low_memory=False)
labels_predict = pd.read_csv("trocafone_kaggle_test.csv", low_memory=False)
features = pd.read_csv('data_set.csv', low_memory = False)

In [7]:
print(training_labels.shape)
training_labels = pd.merge(features, training_labels, on='person', how='inner')
print(training_labels.shape)
training_labels.head(2)

(19414, 2)
(19414, 83)


,person,Email,Organic,Paid,Referral,Social,Unknown,ad campaign hit,brand listing,checkout,...,condition_Bom,condition_Bom - Sem Touch ID,condition_Excelente,condition_Muito Bom,condition_Novo,suma,promedio,maximo,minimo,label
0,0008ed71,0.0,0.0,0.0,2.0,0.0,0.0,0,0,3,...,2.0,0.0,0.0,1.0,0.0,121083102878903,2.018052e+13,20180517162837,20180517122747,0
1,000c79fe,0.0,0.0,1.0,0.0,0.0,0.0,1,0,1,...,4.0,0.0,0.0,0.0,0.0,343068993055724,2.018053e+13,20180529003807,20180529002747,0


In [8]:
print(labels_predict.shape)
labels_predict = pd.merge(features, labels_predict, on='person', how='inner')
print(labels_predict.shape)
labels_predict.head(2)

(19415, 1)
(19415, 82)


,person,Email,Organic,Paid,Referral,Social,Unknown,ad campaign hit,brand listing,checkout,...,storage_8GB,condition_Bom,condition_Bom - Sem Touch ID,condition_Excelente,condition_Muito Bom,condition_Novo,suma,promedio,maximo,minimo
0,00091926,0.0,0.0,8.0,1.0,0.0,0.0,15,25,2,...,0.0,102.0,0.0,108.0,163.0,1.0,9040871528715832,2.018052e+13,20180531195203,20180503220829
1,00091a7a,0.0,0.0,1.0,0.0,0.0,0.0,1,5,0,...,0.0,1.0,0.0,2.0,0.0,0.0,201803261454582,2.018033e+13,20180326145658,20180326145111


In [9]:
train = training_labels
target = 'label'
IDcol = 'person'

In [10]:
predictors = [x for x in train.columns if x not in [target, IDcol]]

In [11]:
cv_params = {'max_depth': [3,5,7], 'min_child_weight': [1,3,5]}
ind_params = {'learning_rate': 0.1, 'n_estimators': 1000, 'seed':0, 'subsample': 0.8, 'colsample_bytree': 0.8, 
             'objective': 'binary:logistic', 'criterion' : 'entropy'}
optimized_GBM = GridSearchCV(xgb.XGBClassifier(**ind_params), 
                            cv_params, 
                             scoring = 'accuracy', cv = 5, n_jobs = -1) 

In [ ]:
optimized_GBM.fit(train[predictors],train[target])

In [12]:
# optimized_GBM.grid_scores_

AttributeError: 'GridSearchCV' object has no attribute 'grid_scores_'

In [13]:
## MEJOR:
## mean: 0.94736, std: 0.00081, params: {'max_depth': 5, 'min_child_weight': 1},

## cambiar esos dos parametros aca:

In [14]:
cv_params = {'learning_rate': [0.1,0.01], 'subsample': [0.7,0.8,0.9]}
ind_params = {'n_estimators': 1000, 'seed':0, 'colsample_bytree': 0.8, 
             'objective': 'binary:logistic', 'criterion' : 'entropy', 'max_depth': 5, 'min_child_weight': 1}
optimized_GBM = GridSearchCV(xgb.XGBClassifier(**ind_params), 
                            cv_params, 
                             scoring = 'accuracy', cv = 5, n_jobs = -1) 

In [ ]:
optimized_GBM.fit(train[predictors],train[target])

In [ ]:
optimized_GBM.grid_scores_

In [ ]:
## MEJOR:
##  mean: 0.94957, std: 0.00128, params: {'learning_rate': 0.01, 'subsample': 0.8},


In [ ]:
### PARAMS: (cambiar abajo)
## {'max_depth': 5, 'min_child_weight': 1},
## {'learning_rate'(ETA): 0.01, 'subsample': 0.8},

In [ ]:
xgdmat = xgb.DMatrix(train[predictors],train[target])

In [ ]:
our_params = {'eta': 0.01, 'seed':0, 'subsample': 0.8, 'colsample_bytree': 0.8, 
             'objective': 'binary:logistic', 'criterion' : 'entropy',
              'max_depth':5, 'min_child_weight':1} 

cv_xgb = xgb.cv(params = our_params, dtrain = xgdmat, num_boost_round = 3000, nfold = 5,
                metrics = ['error'],
                early_stopping_rounds = 100)

In [ ]:
cv_xgb.tail(5)

In [ ]:
our_params = {'eta': 0.1, 'seed':0, 'subsample': 0.8, 'colsample_bytree': 0.8, 
             'objective': 'binary:logistic', 
              'max_depth':3, 'min_child_weight':1} 
## round se saca de arriba 
final_gb = xgb.train(our_params, xgdmat, num_boost_round = 9)

In [ ]:
##########

In [ ]:
%matplotlib inline
import seaborn as sns
sns.set(font_scale = 1.5)

In [ ]:
xgb.plot_importance(final_gb) # fixear

In [ ]:
importances = final_gb.get_fscore()
importances

In [ ]:
#######

In [ ]:
################### FALTA SEPARAR EN TEST Y TRAIN PARA HACER ESTO

In [ ]:
#testdmat = xgb.DMatrix(x_test)

In [ ]:
#from sklearn.metrics import accuracy_score
#y_pred = final_gb.predict(testdmat) # Predict using our testdmat
#y_pred

In [ ]:
#y_pred[y_pred > 0.5] = 1
#y_pred[y_pred <= 0.5] = 0
#y_pred

In [ ]:
#accuracy_score(y_pred, y_test), 1-accuracy_score(y_pred, y_test)

In [ ]:
###########

In [ ]:
set_test = labels_predict
personas = set_test["person"]
set_test = set_test.drop(columns=['person'])
set_test = set_test.fillna(0)

final_test = set_test
testdmat = xgb.DMatrix(final_test)

In [ ]:
y_pred = final_gb.predict(testdmat) # Predict using our testdmat
y_pred

In [ ]:
predicts_kaggel = y_pred.tolist()


In [ ]:
df_kaggel = pd.DataFrame(predicts_kaggel)
df_kaggel.head()

In [ ]:
df_kaggel["person"] = personas
df_kaggel.rename(columns = {0: 'label'},inplace = True) 

In [ ]:
df_kaggel = df_kaggel[['person','label']]

In [ ]:
df_kaggel.to_csv('Resultados_new_xgb.csv',index=False)